In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import sys
import h5py
#sys.path.append('/reg/neh/home/tdd14/tmo_repo') #this is a temporary home for the general TMO analysis code
# sys.path.append('/cds/home/p/paris/reconVMI')
# import analyzers_anna as az # this raises a few runtime errors which require some updating
import psana as ps
from matplotlib import pyplot as plt
from tqdm import tqdm
import matplotlib.colors as colors
from scipy.ndimage import gaussian_filter as gf
from scipy import optimize

from matplotlib import cm
from matplotlib.colors import ListedColormap
from scipy.optimize import minimize
from scipy.signal import find_peaks
import scipy
#from scipy.ndimage import gaussian_filter
sys.path.append('/reg/neh/home/tdd14/modules/cart2pol/cart2pol')
import cart2pol
import scipy.misc as scmis
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import scipy.special as sp
from datetime import datetime
import cv2
sys.path.append('/reg/d/psdm/tmo/tmox51020/results/paris/recon')
from recon_funcs import *

/reg/neh/home/tdd14/modules/CPBASEX/pbasex-Python/pbasex/gData.py:55: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if gData['rBF'] is not 'custom':


Hi! I imported. 


In [4]:
exp = "tmox51020" #"tmoly9120" #'tmox45719' #exp = 'tmolv1720'
gas_jet = 1
runnum = 112
gmd_min = 0.000

mfn = 25 
gfn = 15
gfsig = 0.985 
gfcart = 0.25 
mfcart = 1

In [ ]:
#specs = []
gds = []
tmstmps = []

ds=ps.DataSource(exp = exp, run= runnum)
#ds=ps.DataSource(exp = 'tmox51020', run= runnum, filter = lambda evt:evt.timestamp in tt)

run = next(ds.runs())

#opal1 = run.Detector('tmo_opal1') # OPAL camera looking at the holey mirror
gmd = run.Detector("gmd") # gas intensity detector 
timing = run.Detector('timing') # this gives access to evr codes
ebeam = run.Detector("ebeam")
if exp == "tmox51020": opalfzp = run.Detector('tmo_fzpopal') 
if exp == 'tmolv1720': opalfzp = run.Detector('tmo_opal3') 

n = 0

if hasattr(run, 'epicsinfo'):
    epics_strs = [item[0] for item in run.epicsinfo.keys()][1:] # first one is weird
    epics_detectors = [run.Detector(item) for item in epics_strs]    

for nevent, event in enumerate(run.events()):
    #opal1im_i=np.copy(opal1.raw.raw(event))
    gd_i=gmd.raw.energy(event)
    evr=timing.raw.eventcodes(event)

    # identify relevant event codes
    gas=evr[70] # did the gas jet fire
    ir_timed=evr[67] # was the ir laser timed
    ir_misttimed=evr[68] # was the ir laser mis-timed
    
    if gd_i is None or gd_i<gmd_min or (gas != gas_jet): continue #opal1im_i is None 
    
#     opalfz= np.copy(opalfzp.raw.image(event)) #np.copy(opal1.raw.image(event)) #np.copy(opal1.raw.raw(event))
#     zps_data = opalfz
#     zps_pixel = 100+np.argmax(np.sum(zps_data[100:900],axis=1))
#     px_2d_width = 25 #spacing to cut the 2d image slice to store. Typically 50 pixel width. 25 on either side
#     px_edge_width = 5 #spacing to integrate for the line out. 

#     zps_2d = zps_data[zps_pixel-px_2d_width:zps_pixel+px_2d_width][:]
#     zps = zps_data[zps_pixel-px_edge_width:zps_pixel+px_edge_width][:].mean(0)#
#     zps_bg = zps_data[zps_pixel-px_2d_width:zps_pixel-px_2d_width+px_edge_width][:].mean(0)#
#     zps = zps-zps_bg
#     #spec = np.squeeze(fastsmooth1(zps,10))
#     spec = np.squeeze(zps)
#     spec = np.array(np.squeeze(spec) - np.mean(np.squeeze(spec)[800:900])) 
    
#     specs.append(spec)
    gds.append(gd_i)
    tmstmps.append(event.timestamp)
    n+=1
    if n%50 == 0: print(n)